In [1]:
import os
import requests

import pandas as pd

import requests
import json
from types import SimpleNamespace

import pickle
import os
import re

import openstreetmap_mapping as osm

import pandas as pd
import numpy as np

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool
from bokeh.models import LabelSet
from bokeh.palettes import viridis, Category20, Blues256

output_notebook()

from pyproj import Transformer

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")

In [4]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [5]:

file_name = 'data/kvs'

if os.path.isfile(file_name):
    # open a file, where you stored the pickled data
    file = open(file_name, 'rb')

    # dump information to that file
    kvs = pickle.load(file)

    # close the file
    file.close()

else:
    kvs = osm.toolkit.get_osm_kvs()  
        
    # open a file, where you ant to store the data
    file = open(file_name, 'wb')

    # dump information to that file
    pickle.dump(kvs, file)

    # close the file
    file.close()


In [6]:
tags = Tags(kvs)

In [7]:
area_Pompei = "(40.747,14.481,40.756,14.497)"

In [8]:
df_city_wall = osm.toolkit.get_osm_data(key=tags.historic,tag=tags.historic.citywalls,area=area_Pompei,output="geom",element="way")

for cnt,way in df_city_wall.iterrows():
    city_wall = pd.DataFrame.from_records(way["geometry"])

city_wall["x"],city_wall["y"] = TRANSFORM_4326_TO_3857.transform(city_wall["lat"],city_wall["lon"])

source_city_wall = ColumnDataSource(city_wall)

In [9]:
df_city_gate = osm.toolkit.get_osm_data(key=tags.historic,tag=tags.historic.city_gate,area=area_Pompei,output="center",element="node")

df_city_gate["x"],df_city_gate["y"] = TRANSFORM_4326_TO_3857.transform(df_city_gate["lat"],df_city_gate["lon"])

source_city_gate = ColumnDataSource(df_city_gate[["x","y"]])

In [10]:
df_grass = osm.toolkit.get_osm_data(key=tags.landuse,tag=tags.landuse.grass,area=area_Pompei,output="geom",element="way")

xx = []
yy = []
for cnt,way in df_grass.iterrows():
    geom = pd.DataFrame.from_records(way["geometry"])

    x,y = TRANSFORM_4326_TO_3857.transform(geom["lat"],geom["lon"])

    xx.append(x)
    yy.append(y)

source_grass = ColumnDataSource({"xs":xx,"ys":yy})

In [11]:
df_roads = osm.toolkit.get_osm_data(key=tags.highway,tag=tags.highway.pedestrian,area=area_Pompei,output="geom",element="way")

xx = []
yy = []
for cnt,way in df_roads.iterrows():
    geom = pd.DataFrame.from_records(way["geometry"])

    x,y = TRANSFORM_4326_TO_3857.transform(geom["lat"],geom["lon"])

    xx.append(x)
    yy.append(y)

source_roads = ColumnDataSource({"xs":xx,"ys":yy})

In [12]:
df_building = osm.toolkit.get_osm_data(key=tags.historic,tag=tags.historic.archaeological_site,area=area_Pompei,output="geom",element="way")

xx = []
yy = []
for cnt,way in df_building.iterrows():
    geom = pd.DataFrame.from_records(way["geometry"])

    x,y = TRANSFORM_4326_TO_3857.transform(geom["lat"],geom["lon"])

    xx.append(x)
    yy.append(y)

source_building = ColumnDataSource({"xs":xx,"ys":yy})

In [13]:
df_ruins = osm.toolkit.get_osm_data(key=tags.historic,tag=tags.historic.ruins,area=area_Pompei,output="geom",element="way")

xx = []
yy = []
for cnt,way in df_ruins.iterrows():
    geom = pd.DataFrame.from_records(way["geometry"])

    x,y = TRANSFORM_4326_TO_3857.transform(geom["lat"],geom["lon"])

    xx.append(x)
    yy.append(y)

source_ruins = ColumnDataSource({"xs":xx,"ys":yy})

In [14]:
output_file("Doom.html")

# See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

# Create a bokeh figure with tiles
plot_map = figure(width=800, 
                  height=600,
                  x_axis_type="mercator", 
                  y_axis_type="mercator",
                  match_aspect=True,
                )

plot_map.background_fill_color = "black"
# plot_map.add_tile(MAP_TILES['ESRI'])


In [15]:

plot_map.line(x="x", y="y", source=source_city_wall, color="red")

plot_map.rect(x="x", y="y", source=source_city_gate, width=10, height=10, line_color="yellow", fill_color="yellow")

plot_map.multi_line(xs="xs", ys="ys", source=source_grass, color="peru")

plot_map.multi_line(xs="xs", ys="ys", source=source_building, color="white")

plot_map.multi_line(xs="xs", ys="ys", source=source_ruins, color="yellow")

plot_map.multi_line(xs="xs", ys="ys", source=source_roads, color="red")

plot_map.xgrid.grid_line_color = None
plot_map.ygrid.grid_line_color = None


In [16]:
from bokeh.models import Range1d

# x,y = TRANSFORM_4326_TO_3857.transform([-40,20],[-20,55])

# # set a range using a Range1d
# plot_map.x_range = Range1d(x[0], x[1])
# plot_map.y_range = Range1d(y[0], y[1])

plot_map.title.text = "Pompei's DOOM !"

In [17]:
show(plot_map)